# Lab D.2: Tabular Q-Learning

**Module:** D - Reinforcement Learning (Optional)
**Time:** 1.5-2 hours
**Difficulty:** ⭐⭐⭐☆☆

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand why model-free learning is powerful
- [ ] Implement the Q-learning algorithm from scratch
- [ ] Master exploration vs exploitation with ε-greedy
- [ ] Solve the FrozenLake environment
- [ ] Visualize Q-table evolution during training

---

## 📚 Prerequisites

- Completed: Lab D.1 (Markov Decision Processes)
- Understanding of: Value functions, Bellman equation

---

## 🌍 Real-World Context

**The Problem with Value Iteration:**

In Lab D.1, we needed to know the transition probabilities $P(s'|s,a)$ to solve the MDP. But in the real world:

- 🎮 **Games**: We don't know how the game physics work exactly
- 🤖 **Robotics**: We can't perfectly model friction, slip, sensor noise
- 💬 **LLMs**: We don't know how humans will respond to generated text

**Q-learning's magic:** Learn the optimal policy just by interacting with the environment—no model needed!

> This is called **model-free reinforcement learning**, and it's the foundation of how we train AI systems in complex environments.

---

## 🧒 ELI5: What is Q-Learning?

> **Imagine you're learning to navigate a new city without a map.** 🗺️
>
> Every day, you try different routes to get from home to work:
> - You take a left... and get stuck in traffic (bad!)
> - You take a right... and find a shortcut (good!)
> - Over time, you learn which turns work best
>
> You're building a mental "Q-table" in your head:
> - "At the big intersection, turn RIGHT → usually good"
> - "At the coffee shop, go STRAIGHT → sometimes good"
>
> **The Q-learning update rule is simple:**
> "If I tried something and it turned out better than expected, I should do it more often."
>
> **In AI terms:** Q-learning updates action values based on experience. When we take action $a$ in state $s$, we observe the reward and update our Q-value to be closer to the actual return we received.

---

## Part 1: Setup and Environment

We'll use **Gymnasium** (the successor to OpenAI Gym), specifically the **FrozenLake** environment:

```
SFFF      S: Start
FHFH      F: Frozen (safe)
FFFH      H: Hole (game over!)
HFFG      G: Goal
```

The agent must navigate from S to G without falling in holes!

### Understanding the Gymnasium API

Gymnasium provides a standard interface for RL environments. Here are the key components:

| Component | Description |
|-----------|-------------|
| `gym.make(env_name)` | Creates an environment instance |
| `env.observation_space` | Describes the state space (`.n` gives number of discrete states) |
| `env.action_space` | Describes the action space (`.n` gives number of actions) |
| `env.reset()` | Resets environment, returns `(initial_state, info_dict)` |
| `env.step(action)` | Takes action, returns `(next_state, reward, terminated, truncated, info)` |
| `env.action_space.sample()` | Returns a random valid action |

**Key return values from `env.step()`:**
- `next_state`: The state after taking the action
- `reward`: Immediate reward received
- `terminated`: True if episode ended naturally (goal/failure)
- `truncated`: True if episode was cut short (time limit)
- `info`: Additional diagnostic information (often unused)

**Note on byte strings:** The environment map uses byte strings (e.g., `b'H'` for Hole). To compare, use `== b'H'` or decode with `.decode('utf-8')`.

### Key NumPy Functions for Q-Learning

We'll use these NumPy functions throughout the lab:

- `np.argmax(array)`: Returns the **index** of the maximum value. Essential for finding the best action!
- `np.random.randint(low, high)`: Returns a random integer in range [low, high)
- `np.convolve(array, kernel, mode)`: Used for smoothing data in visualizations

In [ ]:
# Setup - run this first!
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# Install gymnasium if needed
try:
    import gymnasium as gym
except ImportError:
    print("Installing gymnasium...")
    !pip install gymnasium -q
    import gymnasium as gym

np.random.seed(42)

print("🚀 Module D.2: Tabular Q-Learning")
print(f"NumPy version: {np.__version__}")
print(f"Gymnasium version: {gym.__version__}")

In [ ]:
# Create the FrozenLake environment
# is_slippery=False makes it deterministic (easier to learn!)

env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False)

print("🧊 FrozenLake Environment")
print(f"   States: {env.observation_space.n}")
print(f"   Actions: {env.action_space.n}")
print("   Action mapping: 0=Left, 1=Down, 2=Right, 3=Up")
print()

# Visualize the map
print("   Map Layout:")
desc = env.unwrapped.desc
for row in desc:
    print("   " + "".join([c.decode('utf-8') for c in row]))

In [ ]:
# Let's explore the environment

def explore_environment(env, n_episodes: int = 3):
    """Run a few random episodes to understand the dynamics."""
    
    action_names = ['Left', 'Down', 'Right', 'Up']
    
    for episode in range(n_episodes):
        state, _ = env.reset()
        print(f"\n🎮 Episode {episode + 1}:")
        print(f"   Start state: {state}")
        
        total_reward = 0
        steps = []
        
        for step in range(10):  # Max 10 steps
            action = env.action_space.sample()  # Random action
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            steps.append(f"{state}→{action_names[action]}→{next_state}")
            total_reward += reward
            state = next_state
            
            if done:
                if reward == 1:
                    print(f"   🎉 Reached goal in {step + 1} steps!")
                else:
                    print(f"   💀 Fell in hole at step {step + 1}")
                break
        
        print(f"   Path: {' | '.join(steps[:5])}{'...' if len(steps) > 5 else ''}")
        print(f"   Total reward: {total_reward}")

explore_environment(env)

### 🔍 Key Observations

1. **Sparse reward**: Only +1 for reaching the goal, 0 otherwise
2. **Terminal states**: Holes and goal both end the episode
3. **Random policy**: Very unlikely to reach the goal by chance!

We need a smarter approach: **Q-learning**!

---

## Part 2: The Q-Learning Algorithm

### The Update Rule

After taking action $a$ in state $s$, observing reward $r$ and landing in state $s'$:

$$Q(s, a) \leftarrow Q(s, a) + \alpha \left[ r + \gamma \max_{a'} Q(s', a') - Q(s, a) \right]$$

Let's break this down:

| Term | Meaning |
|------|--------|
| $Q(s, a)$ | Current estimate of action value |
| $\alpha$ | Learning rate (how fast we update) |
| $r + \gamma \max Q(s', a')$ | **TD Target**: what we actually got |
| $r + \gamma \max Q(s', a') - Q(s, a)$ | **TD Error**: how wrong we were |

> 🧒 **ELI5**: "I thought this action was worth X, but I actually got Y. Let me adjust my estimate a little bit towards Y."

In [ ]:
class QLearningAgent:
    """
    Tabular Q-Learning Agent.
    
    This agent learns by storing Q-values in a table and updating them
    based on experience using the TD learning rule.
    """
    
    def __init__(self, n_states: int, n_actions: int,
                 learning_rate: float = 0.1,
                 discount_factor: float = 0.99,
                 epsilon_start: float = 1.0,
                 epsilon_min: float = 0.01,
                 epsilon_decay: float = 0.995):
        """
        Initialize the Q-learning agent.
        
        Args:
            n_states: Number of states in the environment
            n_actions: Number of possible actions
            learning_rate: α - how fast we update Q-values
            discount_factor: γ - how much we value future rewards
            epsilon_start: Initial exploration rate
            epsilon_min: Minimum exploration rate
            epsilon_decay: How fast epsilon decreases
        """
        self.n_states = n_states
        self.n_actions = n_actions
        self.lr = learning_rate
        self.gamma = discount_factor
        
        # Exploration parameters
        self.epsilon = epsilon_start
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        
        # Initialize Q-table with zeros
        self.Q = np.zeros((n_states, n_actions))
        
        # For tracking learning progress
        self.td_errors = []
        
        print(f"🤖 Q-Learning Agent Initialized")
        print(f"   Q-table shape: {self.Q.shape}")
        print(f"   Learning rate: {self.lr}")
        print(f"   Discount factor: {self.gamma}")
        print(f"   Initial epsilon: {self.epsilon}")
    
    def select_action(self, state: int, training: bool = True) -> int:
        """
        Select an action using ε-greedy policy.
        
        During training: explore with probability ε
        During evaluation: always exploit (take best action)
        """
        if training and np.random.random() < self.epsilon:
            # Explore: random action
            return np.random.randint(0, self.n_actions)
        else:
            # Exploit: best action according to Q-table
            return np.argmax(self.Q[state])
    
    def update(self, state: int, action: int, reward: float, 
               next_state: int, done: bool) -> float:
        """
        Update Q-value using the Q-learning rule.
        
        Returns the TD error (for tracking).
        """
        # Current Q-value
        current_q = self.Q[state, action]
        
        # TD Target: what we actually observed
        if done:
            # No future rewards after terminal state
            td_target = reward
        else:
            # Reward + discounted best future value
            td_target = reward + self.gamma * np.max(self.Q[next_state])
        
        # TD Error: how wrong our prediction was
        td_error = td_target - current_q
        
        # Update Q-value
        self.Q[state, action] = current_q + self.lr * td_error
        
        # Track for analysis
        self.td_errors.append(abs(td_error))
        
        return td_error
    
    def decay_epsilon(self):
        """Reduce exploration rate over time."""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
    
    def get_policy(self) -> np.ndarray:
        """Extract the greedy policy from Q-table."""
        return np.argmax(self.Q, axis=1)

# Create our agent!
agent = QLearningAgent(
    n_states=env.observation_space.n,
    n_actions=env.action_space.n
)

### ε-Greedy: Exploration vs Exploitation

> 🧒 **ELI5**: Should you go to your favorite restaurant (exploit what you know) or try a new place (explore to learn more)?

- **Too much exploration**: Never commit to good actions
- **Too much exploitation**: Miss better options

ε-greedy is a simple solution:
- With probability ε: take a random action (explore)
- With probability 1-ε: take the best action (exploit)
- Start with high ε (lots of exploration), decay over time

In [ ]:
# Visualize epsilon decay

def plot_epsilon_decay(epsilon_start=1.0, epsilon_min=0.01, epsilon_decay=0.995, n_episodes=1000):
    """Show how epsilon decreases over training."""
    epsilons = []
    epsilon = epsilon_start
    
    for _ in range(n_episodes):
        epsilons.append(epsilon)
        epsilon = max(epsilon_min, epsilon * epsilon_decay)
    
    plt.figure(figsize=(10, 4))
    plt.plot(epsilons)
    plt.xlabel('Episode')
    plt.ylabel('Epsilon (Exploration Rate)')
    plt.title('ε-Greedy Exploration Decay')
    plt.axhline(y=epsilon_min, color='r', linestyle='--', label=f'Min ε = {epsilon_min}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Find when we hit different exploration levels
    for threshold in [0.5, 0.1, 0.05]:
        episode = next((i for i, e in enumerate(epsilons) if e < threshold), n_episodes)
        print(f"   ε < {threshold}: Episode {episode}")

plot_epsilon_decay()

---

## Part 3: Training the Agent

In [ ]:
def train_q_learning(env, agent, n_episodes: int = 2000, 
                     max_steps_per_episode: int = 100,
                     verbose: bool = True) -> Dict:
    """
    Train the Q-learning agent.
    
    Returns training statistics.
    """
    rewards_per_episode = []
    steps_per_episode = []
    success_rate_history = []
    q_table_snapshots = []  # Save Q-table periodically
    
    recent_successes = []
    
    for episode in range(n_episodes):
        state, _ = env.reset()
        total_reward = 0
        steps = 0
        
        for step in range(max_steps_per_episode):
            # Select action
            action = agent.select_action(state, training=True)
            
            # Take action
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            # Update Q-table
            agent.update(state, action, reward, next_state, done)
            
            total_reward += reward
            steps += 1
            state = next_state
            
            if done:
                break
        
        # Decay exploration
        agent.decay_epsilon()
        
        # Track statistics
        rewards_per_episode.append(total_reward)
        steps_per_episode.append(steps)
        recent_successes.append(total_reward > 0)
        
        # Calculate success rate over last 100 episodes
        if len(recent_successes) > 100:
            recent_successes.pop(0)
        success_rate = sum(recent_successes) / len(recent_successes)
        success_rate_history.append(success_rate)
        
        # Save Q-table snapshot
        if episode % 200 == 0:
            q_table_snapshots.append((episode, agent.Q.copy()))
        
        # Print progress
        if verbose and (episode + 1) % 500 == 0:
            print(f"Episode {episode + 1:4d} | "
                  f"Success Rate: {success_rate:.2%} | "
                  f"ε: {agent.epsilon:.3f}")
    
    return {
        'rewards': rewards_per_episode,
        'steps': steps_per_episode,
        'success_rate': success_rate_history,
        'q_snapshots': q_table_snapshots
    }

# Reset agent and train!
agent = QLearningAgent(
    n_states=env.observation_space.n,
    n_actions=env.action_space.n,
    learning_rate=0.8,     # High LR works well for deterministic env
    discount_factor=0.95,
    epsilon_start=1.0,
    epsilon_min=0.01,
    epsilon_decay=0.995
)

print("\n🏋️ Training Q-Learning Agent...\n")
stats = train_q_learning(env, agent, n_episodes=2000)

print("\n✅ Training Complete!")

In [ ]:
# Visualize training progress

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Success rate over time
ax = axes[0]
ax.plot(stats['success_rate'])
ax.set_xlabel('Episode')
ax.set_ylabel('Success Rate (last 100)')
ax.set_title('Learning Progress')
ax.axhline(y=0.7, color='g', linestyle='--', label='Target: 70%')
ax.legend()
ax.grid(True, alpha=0.3)

# Episode length
ax = axes[1]
window = 50
smoothed_steps = np.convolve(stats['steps'], np.ones(window)/window, mode='valid')
ax.plot(smoothed_steps)
ax.set_xlabel('Episode')
ax.set_ylabel('Steps (smoothed)')
ax.set_title('Episode Length')
ax.grid(True, alpha=0.3)

# TD Error
ax = axes[2]
td_errors = agent.td_errors
window = 100
if len(td_errors) > window:
    smoothed_td = np.convolve(td_errors, np.ones(window)/window, mode='valid')
    ax.plot(smoothed_td)
ax.set_xlabel('Update Step')
ax.set_ylabel('|TD Error| (smoothed)')
ax.set_title('TD Error Over Time')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 🔍 What Just Happened?

1. **Success rate** improves over time as the agent learns the optimal path
2. **Episode length** stabilizes as the agent finds efficient routes
3. **TD Error** decreases as Q-values converge to their true values

The agent is learning from experience, without ever being told the rules of the game!

---

## Part 4: Visualizing the Learned Q-Table

In [ ]:
def visualize_q_table(Q: np.ndarray, title: str = "Q-Table"):
    """
    Visualize the Q-table as a heatmap with policy arrows.
    """
    action_symbols = ['←', '↓', '→', '↑']  # Left, Down, Right, Up
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Heatmap of Q-values
    ax = axes[0]
    im = ax.imshow(Q, cmap='RdYlGn', aspect='auto')
    ax.set_xlabel('Action (0=L, 1=D, 2=R, 3=U)')
    ax.set_ylabel('State')
    ax.set_title(f'{title} - Q-Values')
    plt.colorbar(im, ax=ax)
    
    # Grid view with best actions
    ax = axes[1]
    V = np.max(Q, axis=1).reshape(4, 4)
    policy = np.argmax(Q, axis=1).reshape(4, 4)
    
    im = ax.imshow(V, cmap='RdYlGn')
    
    # Add arrows and values
    for i in range(4):
        for j in range(4):
            state = i * 4 + j
            # Check if it's a hole or goal
            if desc[i][j] == b'H':
                ax.text(j, i, '🕳️', ha='center', va='center', fontsize=16)
            elif desc[i][j] == b'G':
                ax.text(j, i, '🎯', ha='center', va='center', fontsize=16)
            else:
                ax.text(j, i - 0.2, action_symbols[policy[i, j]], 
                       ha='center', va='center', fontsize=20, fontweight='bold')
                ax.text(j, i + 0.25, f'{V[i, j]:.2f}', 
                       ha='center', va='center', fontsize=9)
    
    ax.set_title(f'{title} - Optimal Policy')
    ax.set_xticks(range(4))
    ax.set_yticks(range(4))
    ax.grid(True, alpha=0.3)
    plt.colorbar(im, ax=ax, label='State Value')
    
    plt.tight_layout()
    plt.show()

# Show the learned Q-table
visualize_q_table(agent.Q, "Learned")

In [ ]:
# Show Q-table evolution over training

fig, axes = plt.subplots(1, len(stats['q_snapshots']), figsize=(20, 4))

for ax, (episode, Q_snap) in zip(axes, stats['q_snapshots']):
    V = np.max(Q_snap, axis=1).reshape(4, 4)
    im = ax.imshow(V, cmap='RdYlGn', vmin=0, vmax=1)
    ax.set_title(f'Episode {episode}')
    ax.set_xticks([])
    ax.set_yticks([])

plt.suptitle('Q-Table Evolution During Training', fontsize=14)
plt.tight_layout()
plt.show()

---

## Part 5: Evaluating the Learned Policy

In [ ]:
def evaluate_agent(env, agent, n_episodes: int = 100) -> Tuple[float, float]:
    """
    Evaluate the agent's policy (no exploration).
    
    Returns:
        success_rate: Fraction of episodes that reached the goal
        avg_steps: Average steps in successful episodes
    """
    successes = 0
    total_steps_in_successes = 0
    
    for _ in range(n_episodes):
        state, _ = env.reset()
        steps = 0
        
        for step in range(100):
            action = agent.select_action(state, training=False)  # No exploration!
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            steps += 1
            state = next_state
            
            if done:
                if reward > 0:
                    successes += 1
                    total_steps_in_successes += steps
                break
    
    success_rate = successes / n_episodes
    avg_steps = total_steps_in_successes / max(1, successes)
    
    return success_rate, avg_steps

# Evaluate!
success_rate, avg_steps = evaluate_agent(env, agent, n_episodes=1000)

print("\n📊 Evaluation Results (1000 episodes):")
print(f"   Success rate: {success_rate:.1%}")
print(f"   Average steps (successful): {avg_steps:.1f}")

if success_rate >= 0.7:
    print("\n🎉 Achieved >70% success rate! Lab objective complete!")
else:
    print("\n⚠️ Try adjusting hyperparameters or training longer.")

In [ ]:
# Watch a successful episode

def demo_episode(env, agent, verbose: bool = True) -> bool:
    """Run and display one episode."""
    action_names = ['Left', 'Down', 'Right', 'Up']
    
    state, _ = env.reset()
    if verbose:
        print(f"\n🎮 Demo Episode")
        print(f"   Start: State {state}")
    
    path = [state]
    actions = []
    
    for step in range(20):
        action = agent.select_action(state, training=False)
        actions.append(action_names[action])
        
        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        path.append(next_state)
        state = next_state
        
        if done:
            break
    
    if verbose:
        print(f"   Path: {' → '.join(map(str, path))}")
        print(f"   Actions: {' → '.join(actions)}")
        if reward > 0:
            print(f"   🎉 Reached goal in {len(actions)} steps!")
        else:
            print(f"   💀 Failed!")
    
    return reward > 0

# Run a few demos
for i in range(3):
    demo_episode(env, agent)

---

## Part 6: The Slippery Version!

Real FrozenLake has `is_slippery=True`—there's a 33% chance of slipping to an adjacent direction!

This makes learning much harder because:
1. The environment is stochastic
2. Same action can lead to different outcomes
3. Need more exploration to understand the dynamics

In [ ]:
# Create slippery environment
slippery_env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=True)

# Train a new agent with adjusted hyperparameters
slippery_agent = QLearningAgent(
    n_states=slippery_env.observation_space.n,
    n_actions=slippery_env.action_space.n,
    learning_rate=0.1,      # Lower LR for noisy env
    discount_factor=0.99,
    epsilon_start=1.0,
    epsilon_min=0.05,       # Keep some exploration
    epsilon_decay=0.9995    # Slower decay
)

print("\n🏋️ Training on SLIPPERY FrozenLake...\n")
slippery_stats = train_q_learning(slippery_env, slippery_agent, n_episodes=10000)

# Evaluate
success_rate, avg_steps = evaluate_agent(slippery_env, slippery_agent, n_episodes=1000)
print(f"\n📊 Slippery Evaluation:")
print(f"   Success rate: {success_rate:.1%}")
print(f"   (Note: Random policy achieves ~1.5%)")

In [ ]:
# Compare deterministic vs slippery

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

ax = axes[0]
ax.plot(stats['success_rate'], label='Deterministic', alpha=0.8)
ax.plot(slippery_stats['success_rate'], label='Slippery', alpha=0.8)
ax.set_xlabel('Episode')
ax.set_ylabel('Success Rate')
ax.set_title('Learning Curves: Deterministic vs Slippery')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[1]
visualize_q_table(slippery_agent.Q, "Slippery")
plt.show()

### Key Insight

In slippery environments:
- Learning is slower and noisier
- Optimal success rate is lower (you can't always avoid holes)
- The policy must account for uncertainty

This is why RLHF for LLMs is challenging—human preferences are noisy!

---

## ⚠️ Common Mistakes

### Mistake 1: Too High Learning Rate

```python
# ❌ In stochastic environments, high LR causes oscillation
lr = 0.9  # Too high for slippery env!

# ✅ Use lower LR for noisy environments
lr = 0.1  # More stable learning
```

### Mistake 2: Exploration Decay Too Fast

```python
# ❌ Stops exploring before finding good policies
epsilon_decay = 0.9  # Reaches min in ~40 episodes!

# ✅ Decay slowly enough to explore the space
epsilon_decay = 0.995  # Takes ~1000 episodes to reach minimum
```

### Mistake 3: Forgetting Terminal States

```python
# ❌ Using max Q(s') when s' is terminal
td_target = reward + gamma * np.max(Q[next_state])  # Wrong for done=True!

# ✅ No future value for terminal states
if done:
    td_target = reward
else:
    td_target = reward + gamma * np.max(Q[next_state])
```

---

## ✋ Try It Yourself: Hyperparameter Ablation

How do different hyperparameters affect learning? Let's find out!

In [ ]:
# Exercise: Compare different learning rates

def run_ablation(env, learning_rates: list, n_episodes: int = 2000, n_trials: int = 3):
    """Run Q-learning with different learning rates and compare."""
    results = {}
    
    for lr in learning_rates:
        print(f"\n📊 Testing lr = {lr}...")
        trial_success_rates = []
        
        for trial in range(n_trials):
            agent = QLearningAgent(
                n_states=env.observation_space.n,
                n_actions=env.action_space.n,
                learning_rate=lr,
                discount_factor=0.99,
                epsilon_start=1.0,
                epsilon_min=0.01,
                epsilon_decay=0.995
            )
            
            # Train silently
            _ = train_q_learning(env, agent, n_episodes=n_episodes, verbose=False)
            
            # Evaluate
            success_rate, _ = evaluate_agent(env, agent, n_episodes=500)
            trial_success_rates.append(success_rate)
        
        results[lr] = {
            'mean': np.mean(trial_success_rates),
            'std': np.std(trial_success_rates)
        }
        print(f"   Success rate: {results[lr]['mean']:.1%} ± {results[lr]['std']:.1%}")
    
    return results

# Run ablation (this will take a few minutes)
learning_rates = [0.01, 0.1, 0.5, 0.9]
ablation_results = run_ablation(env, learning_rates, n_episodes=1500, n_trials=3)

# Visualize
plt.figure(figsize=(8, 5))
lrs = list(ablation_results.keys())
means = [ablation_results[lr]['mean'] for lr in lrs]
stds = [ablation_results[lr]['std'] for lr in lrs]

plt.bar(range(len(lrs)), means, yerr=stds, capsize=5, alpha=0.7)
plt.xticks(range(len(lrs)), [str(lr) for lr in lrs])
plt.xlabel('Learning Rate')
plt.ylabel('Success Rate')
plt.title('Q-Learning: Effect of Learning Rate')
plt.axhline(y=0.7, color='g', linestyle='--', label='Target')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

## 🎉 Checkpoint

You've learned:
- ✅ **Model-free RL**: Learning without knowing transition probabilities
- ✅ **Q-learning algorithm**: Update Q-values from experience
- ✅ **TD Learning**: Bootstrap estimates from current values
- ✅ **ε-Greedy**: Balance exploration and exploitation
- ✅ **Hyperparameter tuning**: Learning rate, discount, exploration decay

---

## 🚀 Challenge: Double Q-Learning

Standard Q-learning tends to **overestimate** Q-values because it uses the same Q-table to both select and evaluate actions.

**Double Q-learning** uses two Q-tables:
1. Use Q1 to select the best action
2. Use Q2 to evaluate that action (and vice versa)

Can you implement it?

In [ ]:
# Challenge: Implement Double Q-Learning

class DoubleQLearningAgent:
    """
    Double Q-Learning to reduce overestimation bias.
    """
    
    def __init__(self, n_states: int, n_actions: int, lr: float = 0.1, gamma: float = 0.99):
        self.n_states = n_states
        self.n_actions = n_actions
        self.lr = lr
        self.gamma = gamma
        
        # Two Q-tables!
        self.Q1 = np.zeros((n_states, n_actions))
        self.Q2 = np.zeros((n_states, n_actions))
        
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
    
    def select_action(self, state: int, training: bool = True) -> int:
        if training and np.random.random() < self.epsilon:
            return np.random.randint(0, self.n_actions)
        # Use sum of both Q-tables for action selection
        return np.argmax(self.Q1[state] + self.Q2[state])
    
    def update(self, state: int, action: int, reward: float, 
               next_state: int, done: bool):
        """
        Double Q-learning update.
        
        Randomly update Q1 or Q2, using the other for evaluation.
        """
        if np.random.random() < 0.5:
            # Update Q1, use Q2 for evaluation
            if done:
                target = reward
            else:
                # Select best action using Q1, but evaluate with Q2
                best_action = np.argmax(self.Q1[next_state])
                target = reward + self.gamma * self.Q2[next_state, best_action]
            
            self.Q1[state, action] += self.lr * (target - self.Q1[state, action])
        else:
            # Update Q2, use Q1 for evaluation
            if done:
                target = reward
            else:
                best_action = np.argmax(self.Q2[next_state])
                target = reward + self.gamma * self.Q1[next_state, best_action]
            
            self.Q2[state, action] += self.lr * (target - self.Q2[state, action])
    
    def decay_epsilon(self):
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)
    
    @property
    def Q(self):
        """Combined Q-table for evaluation."""
        return (self.Q1 + self.Q2) / 2

# Test Double Q-Learning
double_agent = DoubleQLearningAgent(
    n_states=slippery_env.observation_space.n,
    n_actions=slippery_env.action_space.n,
    lr=0.1,
    gamma=0.99
)

print("\n🏋️ Training Double Q-Learning on Slippery FrozenLake...")

# Custom training loop for DoubleQLearning
for episode in range(10000):
    state, _ = slippery_env.reset()
    
    for step in range(100):
        action = double_agent.select_action(state)
        next_state, reward, terminated, truncated, _ = slippery_env.step(action)
        done = terminated or truncated
        double_agent.update(state, action, reward, next_state, done)
        state = next_state
        if done:
            break
    
    double_agent.decay_epsilon()

# Evaluate
successes = 0
for _ in range(1000):
    state, _ = slippery_env.reset()
    for _ in range(100):
        action = double_agent.select_action(state, training=False)
        state, reward, terminated, truncated, _ = slippery_env.step(action)
        if terminated or truncated:
            successes += reward
            break

print(f"\n📊 Double Q-Learning Success Rate: {successes/10:.1%}")

---

## 🔗 Connection to Deep Learning

Tabular Q-learning works great for small state spaces (16 states in FrozenLake).

But what about:
- **Atari games**: 210 × 160 pixels × 128 colors = ~10^67 possible states!
- **LLMs**: The state is the entire context window (potentially infinite!)

**Solution: Use neural networks to approximate Q(s, a)**

This is **Deep Q-Learning (DQN)**, which we'll cover in the next lab!

---

## 📖 Further Reading

- [Sutton & Barto, Chapter 6](http://incompleteideas.net/book/the-book-2nd.html) - Temporal-Difference Learning
- [Q-learning Wikipedia](https://en.wikipedia.org/wiki/Q-learning)
- [CleanRL Q-learning](https://github.com/vwxyzjn/cleanrl) - Production implementations

---

## 🧹 Cleanup

In [ ]:
# Close environments
env.close()
slippery_env.close()

import gc
gc.collect()

print("✅ Notebook complete! Ready for Lab D.3: Deep Q-Networks")